In [131]:
import pandas as pd
import re

# Shows more characters in the output when you print data
pd.options.display.max_colwidth = 100

def format_by_transaction_code(dataframe):
    col_names=['Organisation','Transaction_Date','IsContactless','Other_info','Other_info_2']
    formatted_dataframe = pd.DataFrame(columns=col_names)
    for index, code in enumerate(dataframe['Transaction Code']):
        match code:
            case 'POS':
                length = len(dataframe.loc[index,'Transaction_Description'].split(','))
                df = pd.DataFrame({dataframe.loc[index,'Transaction_Description']})

                if length == 2:
                    df = pd.DataFrame({dataframe.loc[index,'Transaction_Description']})
                    df[['Card_Date_Contactless','Organisation']] = df[0].str.split(',', expand=True)
                    if len(df['Card_Date_Contactless'].str.split()[0]) == 2:

                        df[['Card_Num','Transaction_Date']] = df['Card_Date_Contactless'].str.split(expand=True)
                    else:
                        df[['Card_Num','Transaction_Date','IsContactless']] = df['Card_Date_Contactless'].str.split(expand=True)
                    df.drop(columns=[0], inplace=True)
                    df.drop(columns=['Card_Date_Contactless'], inplace=True)
                    df.drop(columns=['Card_Num'], inplace=True)

                if length == 3:
                    df = pd.DataFrame({dataframe.loc[index,'Transaction_Description']})
                    df[['Card_Date_Contactless','Organisation','Other_info']] = df[0].str.split(',', expand=True)
                    if len(df['Card_Date_Contactless'].str.split()[0]) == 2:

                        df[['Card_Num','Transaction_Date']] = df['Card_Date_Contactless'].str.split(expand=True)
                    else:
                        df[['Card_Num','Transaction_Date','IsContactless']] = df['Card_Date_Contactless'].str.split(expand=True)
                    df.drop(columns=[0], inplace=True)
                    df.drop(columns=['Card_Date_Contactless'], inplace=True)
                    df.drop(columns=['Card_Num'], inplace=True)

                if length == 4:
                    df = pd.DataFrame({dataframe.loc[index,'Transaction_Description']})
                    df[['Card_Date_Contactless','Organisation','Other_info','Other_info_2']] = df[0].str.split(',', expand=True)
                    if len(df['Card_Date_Contactless'].str.split()[0]) == 2:

                        df[['Card_Num','Transaction_Date']] = df['Card_Date_Contactless'].str.split(expand=True)
                    else:
                        df[['Card_Num','Transaction_Date','IsContactless']] = df['Card_Date_Contactless'].str.split(expand=True)
                    df.drop(columns=[0], inplace=True)
                    df.drop(columns=['Card_Date_Contactless'], inplace=True)
                    df.drop(columns=['Card_Num'], inplace=True)

                # Add to master Dataframe
                formatted_dataframe = formatted_dataframe.append(df, ignore_index = True)
            case 'BAC':
                pass
                #print(dataframe.loc[index,:])
            case 'DPC':
                pass
                #print(dataframe.loc[index,:])
            case _:
                return "Sorry, we arent able to format this transaction type yet"
    print(formatted_dataframe)
    return formatted_dataframe

def format_data(user_id):
    # Reads in the Transaction and Balance Json files from Nordigen, Drops the Transaction ID column since it has
    # useless information
    df = pd.read_json(fr"C:\Users\Jack\Documents\GitHub\NetWorth\Bank Account Transactions\Formatting\User Transaction Data\{user_id}_Transactions.json")

    # Pending transactions does not sync with the balance in Nordigen, as a result we are adding the pending
    # transactions to our booked transactions to compensate and show the proper balance.
    df_transactions = pd.DataFrame(df['transactions']['booked'])
    df_transactions_pending = pd.DataFrame(df['transactions']['pending'])

    df_transactions = pd.concat([df_transactions, df_transactions_pending])
    df_transactions["bookingDate"] = pd.to_datetime(df_transactions["bookingDate"])
    df_transactions.reset_index(inplace=True, drop=True)

    df_transactionamount = pd.json_normalize(df_transactions['transactionAmount'])
    df_transactions = pd.concat([df_transactions, df_transactionamount], axis=1).drop('transactionAmount', axis = 1)
    df_transactions.drop(columns=['transactionId'], inplace=True)
    df_transactions = df_transactions.sort_values(by='bookingDate', ascending=False)
    df_transactions.reset_index(inplace=True, drop=True)

    df = pd.read_json(fr"C:\Users\Jack\Documents\GitHub\NetWorth\Bank Account Transactions\Formatting\User Transaction Data\Testid_00000009_Balances.json")
    # Since we will use booked transactions from the "Transactions" data, we only want to see the forwardavailable
    # balance from the "Balance" data
    df_balance = pd.DataFrame(df['balances'][1])

    # This creates a balance column using the current balance in the "balance" data taken from the Nordigen API
    current_balance = []
    value = float(df_balance['balanceAmount'].loc['amount'])
    for balance in df_transactions['amount']:
        current_balance.append(value)
        value -= float(balance)
    df_transactions['balance'] = current_balance

    # This removed all the unexpected whitespace, that is longer than expected, reducing them to a specified size, '  ',
    # this space is then replaced by ',' so that it can be separated by the comma later when creating new columns.
    finished = False
    while not finished:
        counter = 0
        for index, description in enumerate(df_transactions['remittanceInformationUnstructured']):
            Tokens = re.findall('\s+', description)
            for token in Tokens:
                if len(token) > len('  '):
                    df_transactions.loc[index,'remittanceInformationUnstructured'] = df_transactions.loc[index,'remittanceInformationUnstructured'].replace(' '*len(token),'  ')
                    counter += 1
        if counter == 0:
            finished = True
            print("Finished!")
        else:
            print("Needs another go!")
    df_transactions['remittanceInformationUnstructured'] = df_transactions['remittanceInformationUnstructured'].str.replace('  ',',').str.strip()

    df = pd.read_csv('Transaction Explanations.txt',sep = ';')
    df_transactions.rename(columns = {'proprietaryBankTransactionCode':'Transaction Code'}, inplace = True)
    df_transactions.rename(columns = {'remittanceInformationUnstructured':'Transaction_Description'}, inplace = True)
    df_transactions = df_transactions.merge(df, on ='Transaction Code', how='left')

    format_by_transaction_code(df_transactions)
format_data("Testid_00000009")

Needs another go!
Needs another go!
Needs another go!
Finished!
                    Organisation Transaction_Date IsContactless  \
0                  TFL TRAVEL CH          26NOV21             C   
1                      ZTL*HOSHI          26NOV21             C   
2                  TFL TRAVEL CH          18NOV21             C   
3                    BEIGEL BAKE          18NOV21             C   
4                  TFL TRAVEL CH          16NOV21             C   
5                  TFL TRAVEL CH          12NOV21             C   
6                  TFL TRAVEL CH          12NOV21             C   
7                            HRK          13NOV21           NaN   
8                      BOULEVARD          12NOV21           NaN   
9                      19 20 BAR          11NOV21             C   
10                 TFL TRAVEL CH          11NOV21             C   
11                     19 20 BAR          11NOV21             C   
12                     19 20 BAR          11NOV21             C  